In [3]:
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from nba_api.stats.endpoints import leaguegamefinder, boxscoreadvancedv3
from nba_api.stats.static import teams

In [233]:
nba_teams = pd.DataFrame(teams.get_teams())
nba_teams = dict(zip(nba_teams['abbreviation'], nba_teams['id']))
game_schedules = pd.DataFrame()
for i in range(24):
    year_start = '0' + str(i) if i < 10 else str(i)
    year_end = '0' + str(i + 1) if i < 10 else str(i + 1)
    for team, team_id, in tqdm(nba_teams.items()):
        team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=f'20{year_start}-{year_end}')
        game_schedules = pd.concat([game_schedules, team_games.get_data_frames()[0]])
game_schedules

  0%|          | 0/30 [00:00<?, ?it/s]

/var/folders/xk/d3trzxnn5nv63q9frbz67bs40000gn/T/ipykernel_15313/578778309.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  game_schedules = pd.concat([game_schedules, team_games.get_data_frames()[0]])


  0%|          | 0/30 [00:00<?, ?it/s]

/var/folders/xk/d3trzxnn5nv63q9frbz67bs40000gn/T/ipykernel_15313/578778309.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  game_schedules = pd.concat([game_schedules, team_games.get_data_frames()[0]])


  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22000,1610612737,ATL,Atlanta Hawks,0020001172,2001-04-17,ATL vs. WAS,W,240,122,...,0.686,19,26,45,22,10,5,13,26,3.0
1,22000,1610612737,ATL,Atlanta Hawks,0020001165,2001-04-16,ATL @ CHH,L,239,83,...,0.688,11,26,37,22,10,2,17,21,-23.0
2,22000,1610612737,ATL,Atlanta Hawks,0020001149,2001-04-14,ATL vs. DET,L,265,124,...,0.763,15,35,50,29,11,5,18,36,-5.0
3,22000,1610612737,ATL,Atlanta Hawks,0020001138,2001-04-13,ATL @ NYK,L,240,65,...,0.917,11,23,34,15,7,3,21,19,-36.0
4,22000,1610612737,ATL,Atlanta Hawks,0020001127,2001-04-11,ATL vs. CLE,W,240,107,...,0.783,17,24,41,26,8,7,14,29,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,22023,1610612766,CHA,Charlotte Hornets,0022300063,2023-10-25,CHA vs. ATL,W,240,116,...,0.731,12,39,51,34,5,3,19,21,6.0
36,12023,1610612766,CHA,Charlotte Hornets,0012300060,2023-10-19,CHA vs. BOS,L,241,99,...,0.808,10,39,49,24,8,5,24,17,-28.0
37,12023,1610612766,CHA,Charlotte Hornets,0012300038,2023-10-15,CHA vs. OKC,W,241,117,...,0.591,8,35,43,28,10,7,12,15,2.0
38,12023,1610612766,CHA,Charlotte Hornets,0012300025,2023-10-12,CHA @ WAS,L,241,92,...,0.667,16,50,66,19,9,9,23,24,-6.0


In [234]:
# Don't run twice, or else you have to rerun above cell
cleaned_games = game_schedules
cleaned_games['IS_HOME_TEAM'] = cleaned_games['MATCHUP'].str.contains('vs.').astype(int)
cleaned_games['MATCHUP'] = cleaned_games['MATCHUP'].str.replace('@', 'vs.')
cleaned_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,IS_HOME_TEAM
0,22000,1610612737,ATL,Atlanta Hawks,0020001172,2001-04-17,ATL vs. WAS,W,240,122,...,19,26,45,22,10,5,13,26,3.0,1
1,22000,1610612737,ATL,Atlanta Hawks,0020001165,2001-04-16,ATL vs. CHH,L,239,83,...,11,26,37,22,10,2,17,21,-23.0,0
2,22000,1610612737,ATL,Atlanta Hawks,0020001149,2001-04-14,ATL vs. DET,L,265,124,...,15,35,50,29,11,5,18,36,-5.0,1
3,22000,1610612737,ATL,Atlanta Hawks,0020001138,2001-04-13,ATL vs. NYK,L,240,65,...,11,23,34,15,7,3,21,19,-36.0,0
4,22000,1610612737,ATL,Atlanta Hawks,0020001127,2001-04-11,ATL vs. CLE,W,240,107,...,17,24,41,26,8,7,14,29,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,22023,1610612766,CHA,Charlotte Hornets,0022300063,2023-10-25,CHA vs. ATL,W,240,116,...,12,39,51,34,5,3,19,21,6.0,1
36,12023,1610612766,CHA,Charlotte Hornets,0012300060,2023-10-19,CHA vs. BOS,L,241,99,...,10,39,49,24,8,5,24,17,-28.0,1
37,12023,1610612766,CHA,Charlotte Hornets,0012300038,2023-10-15,CHA vs. OKC,W,241,117,...,8,35,43,28,10,7,12,15,2.0,1
38,12023,1610612766,CHA,Charlotte Hornets,0012300025,2023-10-12,CHA vs. WAS,L,241,92,...,16,50,66,19,9,9,23,24,-6.0,0


In [235]:
team_stats = cleaned_games.drop(['SEASON_ID', 'TEAM_NAME'], axis=1)
team_stats = team_stats.set_index(['GAME_DATE'])
team_stats['OPPONENT'] = team_stats['MATCHUP'].str[-3:]
masking_dict = dict(zip(team_stats['TEAM_ABBREVIATION'], team_stats['TEAM_ID']))
team_stats['OPPONENT'].replace(masking_dict, inplace=True)
team_stats = team_stats[team_stats['OPPONENT'].isin(masking_dict.values())]
team_stats

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,IS_HOME_TEAM,OPPONENT
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2001-04-17,1610612737,ATL,0020001172,ATL vs. WAS,W,240,122,45,99,0.455,...,26,45,22,10,5,13,26,3.0,1,1610612764
2001-04-16,1610612737,ATL,0020001165,ATL vs. CHH,L,239,83,34,85,0.400,...,26,37,22,10,2,17,21,-23.0,0,1610612766
2001-04-14,1610612737,ATL,0020001149,ATL vs. DET,L,265,124,46,97,0.474,...,35,50,29,11,5,18,36,-5.0,1,1610612765
2001-04-13,1610612737,ATL,0020001138,ATL vs. NYK,L,240,65,26,71,0.366,...,23,34,15,7,3,21,19,-36.0,0,1610612752
2001-04-11,1610612737,ATL,0020001127,ATL vs. CLE,W,240,107,42,86,0.488,...,24,41,26,8,7,14,29,5.0,1,1610612739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,1610612766,CHA,0022300063,CHA vs. ATL,W,240,116,43,86,0.500,...,39,51,34,5,3,19,21,6.0,1,1610612737
2023-10-19,1610612766,CHA,0012300060,CHA vs. BOS,L,241,99,37,88,0.420,...,39,49,24,8,5,24,17,-28.0,1,1610612738
2023-10-15,1610612766,CHA,0012300038,CHA vs. OKC,W,241,117,46,88,0.523,...,35,43,28,10,7,12,15,2.0,1,1610612760


In [236]:
model_data = team_stats.drop(['TEAM_ABBREVIATION', 'MATCHUP', 'WL'], axis=1)
model_data

,TEAM_ID,GAME_ID,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,IS_HOME_TEAM,OPPONENT
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2001-04-17,1610612737,0020001172,240,122,45,99,0.455,8,20,0.400,...,26,45,22,10,5,13,26,3.0,1,1610612764
2001-04-16,1610612737,0020001165,239,83,34,85,0.400,4,12,0.333,...,26,37,22,10,2,17,21,-23.0,0,1610612766
2001-04-14,1610612737,0020001149,265,124,46,97,0.474,3,14,0.214,...,35,50,29,11,5,18,36,-5.0,1,1610612765
2001-04-13,1610612737,0020001138,240,65,26,71,0.366,2,10,0.200,...,23,34,15,7,3,21,19,-36.0,0,1610612752
2001-04-11,1610612737,0020001127,240,107,42,86,0.488,5,14,0.357,...,24,41,26,8,7,14,29,5.0,1,1610612739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,1610612766,0022300063,240,116,43,86,0.500,11,37,0.297,...,39,51,34,5,3,19,21,6.0,1,1610612737
2023-10-19,1610612766,0012300060,241,99,37,88,0.420,4,28,0.143,...,39,49,24,8,5,24,17,-28.0,1,1610612738
2023-10-15,1610612766,0012300038,241,117,46,88,0.523,12,38,0.316,...,35,43,28,10,7,12,15,2.0,1,1610612760


In [237]:
original = model_data.reset_index().set_index(['GAME_DATE', 'TEAM_ID']).sort_index(level=[1, 0])
twisted_team_1 = original
for i in range(1, 6):
    columns = [f'{column}_past_{i}' for column in original.columns]
    shifted = original.groupby('TEAM_ID').shift(i)
    shifted.columns = columns
    twisted_team_1 = pd.concat([twisted_team_1, shifted], axis=1)
twisted_team_1.drop([column for column in original if column not in ['TEAM_ID', 'GAME_ID', 'OPPONENT', 'PLUS_MINUS']], inplace=True, axis=1)
twisted_team_1.reset_index(inplace=True)
twisted_team_1

,GAME_DATE,TEAM_ID,GAME_ID,PLUS_MINUS,OPPONENT,GAME_ID_past_1,MIN_past_1,PTS_past_1,FGM_past_1,FGA_past_1,...,DREB_past_5,REB_past_5,AST_past_5,STL_past_5,BLK_past_5,TOV_past_5,PF_past_5,PLUS_MINUS_past_5,IS_HOME_TEAM_past_5,OPPONENT_past_5
0,2000-10-31,1610612737,0020000004,-24.0,1610612766,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-11-02,1610612737,0020000021,-25.0,1610612752,0020000004,241,82,30,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-11-04,1610612737,0020000036,-3.0,1610612753,0020000021,239,69,26,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-11-06,1610612737,0020000054,-10.0,1610612763,0020000036,240,104,39,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-11-07,1610612737,0020000060,-9.0,1610612757,0020000054,241,87,32,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60749,2024-01-02,1610612766,0022300464,7.0,1610612758,0022300455,240,93,36,86,...,34,47,23,8,11,17,15,-7.0,1.0,1610612743
60750,2024-01-05,1610612766,0022300484,-13.0,1610612741,0022300464,239,111,42,82,...,34,42,31,8,6,11,23,-9.0,0.0,1610612746
60751,2024-01-08,1610612766,0022300506,-7.0,1610612741,0022300484,240,91,36,89,...,35,43,29,5,5,19,14,-21.0,0.0,1610612747
60752,2024-01-10,1610612766,0022300518,-25.0,1610612758,0022300506,265,112,40,94,...,18,26,32,5,3,6,17,-14.0,0.0,1610612756


In [238]:
original = model_data.reset_index().set_index(['GAME_DATE', 'OPPONENT']).sort_index(level=[1, 0])
twisted_team_2 = original
for i in range(1, 6):
    columns = [f'{column}_past_{i}' for column in original.columns]
    shifted = original.groupby('OPPONENT').shift(i)
    shifted.columns = columns
    twisted_team_2 = pd.concat([twisted_team_2, shifted], axis=1)
twisted_team_2.drop([column for column in original if column not in ['TEAM_ID', 'GAME_ID', 'OPPONENT', 'MATCHUP', 'PLUS_MINUS']], inplace=True, axis=1)
twisted_team_2.reset_index(inplace=True)

In [239]:
merged_data = twisted_team_1.merge(twisted_team_2, on=['GAME_DATE', 'GAME_ID'], how='inner', suffixes=['_team', '_op'])
merged_data = merged_data.drop_duplicates('GAME_ID').set_index(['GAME_DATE', 'GAME_ID'])
merged_data.drop(['PLUS_MINUS_op', 'OPPONENT_op', 'TEAM_ID_op'], axis=1, inplace=True)
merged_data.dropna(inplace=True)
merged_data

,,TEAM_ID_team,PLUS_MINUS_team,OPPONENT_team,GAME_ID_past_1_team,MIN_past_1_team,PTS_past_1_team,FGM_past_1_team,FGA_past_1_team,FG_PCT_past_1_team,FG3M_past_1_team,...,OREB_past_5_op,DREB_past_5_op,REB_past_5_op,AST_past_5_op,STL_past_5_op,BLK_past_5_op,TOV_past_5_op,PF_past_5_op,PLUS_MINUS_past_5_op,IS_HOME_TEAM_past_5_op
GAME_DATE,GAME_ID,,,,,,,,,,,,,,,,,,,,,
2000-11-09,0020000077,1610612737,-9.0,1610612756,0020000060,241,88,31,84,0.369,3,...,7,37,44,16,7,6,17,23,24.0,0.0
2000-11-10,0020000084,1610612737,-9.0,1610612746,0020000077,240,79,33,81,0.407,2,...,2,34,36,22,4,5,12,21,25.0,1.0
2000-11-14,0020000104,1610612737,2.0,1610612757,0020000084,240,106,37,70,0.529,6,...,15,25,40,14,5,6,22,23,3.0,0.0
2000-11-15,0020000115,1610612737,-10.0,1610612749,0020000104,241,99,37,75,0.493,6,...,14,33,47,20,11,4,14,22,10.0,1.0
2000-11-17,0020000126,1610612737,-7.0,1610612738,0020000115,242,74,27,69,0.391,5,...,10,33,43,27,6,15,18,25,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-14,0022200414,1610612765,7.0,1610612766,0022200396,240,117,43,93,0.462,17,...,12,34,46,28,9,5,13,21,10.0,0.0
2023-02-03,0022200785,1610612765,6.0,1610612766,0022200761,240,105,36,79,0.456,14,...,11,37,48,23,11,8,14,17,18.0,0.0
2023-02-27,0022200919,1610612765,-11.0,1610612766,0022200903,240,91,38,94,0.404,6,...,13,38,51,21,7,3,17,32,-7.0,0.0


In [240]:
from sklearn.model_selection import train_test_split
target_column = 'PLUS_MINUS_team'
train_columns = [column for column in merged_data.columns if column != target_column]
X = merged_data[train_columns]
y = merged_data[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=None)
X_train.to_csv('test.csv')

In [242]:
from sklearn.linear_model import Ridge
clf = Ridge(random_state=42).fit(X_train, y_train)
clf.score(X_test, y_test)

0.04000848807879731

In [155]:
home = team_stats[team_stats['IS_HOME_TEAM'] == 1]
away = team_stats[team_stats['IS_HOME_TEAM'] == 0]

In [25]:
merged = home.merge(away, on=['GAME_DATE','GAME_ID'], how='inner')
merged

,TEAM_ID_x,TEAM_ABBREVIATION_x,GAME_ID,MATCHUP_x,WL_x,MIN_x,PTS_x,FGM_x,FGA_x,FG_PCT_x,...,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PLUS_MINUS_y,IS_HOME_TEAM_y,OPPONENT_y
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2024-01-03,1610612737,ATL,0022300467,ATL vs. OKC,W,240,141,48,95,0.505,...,28,37,30,10,1,13,23,-3.0,0,ATL
2023-12-29,1610612737,ATL,0022300431,ATL vs. SAC,L,239,110,37,94,0.394,...,33,38,35,9,3,12,20,7.0,0,ATL
2023-12-23,1610612737,ATL,0022300393,ATL vs. MEM,L,239,119,45,96,0.469,...,36,51,29,6,7,11,16,6.0,0,ATL
2023-12-18,1610612737,ATL,0022300352,ATL vs. DET,W,240,130,46,85,0.541,...,24,36,23,7,3,13,26,-6.0,0,ATL
2023-12-11,1610612737,ATL,0022300296,ATL vs. DEN,L,240,122,44,96,0.458,...,35,49,29,9,8,16,19,7.0,0,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-30,1610612766,CHA,0022300101,CHA vs. BKN,L,240,121,47,96,0.490,...,36,46,25,4,8,13,19,12.0,0,CHA
2023-10-27,1610612766,CHA,0022300077,CHA vs. DET,L,240,99,33,88,0.375,...,41,53,28,7,6,23,29,12.0,0,CHA
2023-10-25,1610612766,CHA,0022300063,CHA vs. ATL,W,240,116,43,86,0.500,...,30,42,24,12,1,12,19,-6.0,0,CHA


In [51]:
team_1_twisted = merged
team_1_twisted = team_1_twisted.reset_index().set_index(['GAME_DATE', 'TEAM_ABBREVIATION_X'])


,,TEAM_ID_x,GAME_ID,MATCHUP_x,WL_x,MIN_x,PTS_x,FGM_x,FGA_x,FG_PCT_x,FG3M_x,...,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PLUS_MINUS_y,IS_HOME_TEAM_y,OPPONENT_y
GAME_DATE,TEAM_ABBREVIATION_x,,,,,,,,,,,,,,,,,,,,,
2024-01-03,ATL,1610612737,0022300467,ATL vs. OKC,W,240,141,48,95,0.505,14,...,28,37,30,10,1,13,23,-3.0,0,ATL
2023-12-29,ATL,1610612737,0022300431,ATL vs. SAC,L,239,110,37,94,0.394,15,...,33,38,35,9,3,12,20,7.0,0,ATL
2023-12-23,ATL,1610612737,0022300393,ATL vs. MEM,L,239,119,45,96,0.469,13,...,36,51,29,6,7,11,16,6.0,0,ATL
2023-12-18,ATL,1610612737,0022300352,ATL vs. DET,W,240,130,46,85,0.541,16,...,24,36,23,7,3,13,26,-6.0,0,ATL
2023-12-11,ATL,1610612737,0022300296,ATL vs. DEN,L,240,122,44,96,0.458,17,...,35,49,29,9,8,16,19,7.0,0,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-30,CHA,1610612766,0022300101,CHA vs. BKN,L,240,121,47,96,0.490,8,...,36,46,25,4,8,13,19,12.0,0,CHA
2023-10-27,CHA,1610612766,0022300077,CHA vs. DET,L,240,99,33,88,0.375,7,...,41,53,28,7,6,23,29,12.0,0,CHA
2023-10-25,CHA,1610612766,0022300063,CHA vs. ATL,W,240,116,43,86,0.500,11,...,30,42,24,12,1,12,19,-6.0,0,CHA


In [245]:
box_scores = pd.DataFrame()
for game_id in tqdm(merged_data.index.get_level_values('GAME_ID').unique()):
    game_box_scores = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id)
    box_scores = pd.concat([box_scores, game_box_scores.get_data_frames()[0]])
box_scores.to_csv('box_scores.csv')

  0%|          | 0/29716 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [243]:
indexed = merged_data.reset_index().set_index(['GAME_DATE', 'GAME_ID'])
past_games = indexed
for i in range(4):
    columns = [column + '_past' + str(i) for column in merged.columns]
    past_games[columns] = merged.shift(i)
past_games

ValueError: cannot reindex on an axis with duplicate labels